In [3]:
CPD_URL = "https://cpd-cpd.ai-governance-94074a334e51addd457c5646c0f9a073-0000.us-east.containers.appdomain.cloud"
CPD_USERNAME = "admin"
CPD_PASSWORD = "cp4d@dm1n"

CPD_URL = CPD_URL.rstrip("/")

WML_CREDENTIALS = {
    "url": CPD_URL,
    "password": CPD_PASSWORD,
    "username": CPD_USERNAME,
    "instance_id": "wml_local",
    "version": "4.5"
}

SPACE_ID = "81ec139c-bd3b-4028-a8c4-70ef8aa25193"

ROWS_TO_SCORE = 10

In [2]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
print(wml_client.version)

1.0.256


In [4]:
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [5]:
wml_client.deployments.list()

------------------------------------  -------------------------------  -----  ------------------------
GUID                                  NAME                             STATE  CREATED
03f33cc8-f4d3-43e2-b586-98c0ec5c9f16  Employee attrition - challenger  ready  2023-04-05T19:55:40.838Z
------------------------------------  -------------------------------  -----  ------------------------


In [6]:
MODEL_UID = "03f33cc8-f4d3-43e2-b586-98c0ec5c9f16"

In [7]:
import itc_utils.flight_service as itcfs

readClient = itcfs.get_flight_client()

nb_data_request = {
    'data_name': """modeling_records_2022.csv""",
    'interaction_properties': {
        #'row_limit': 500,
        'infer_schema': 'false',
        'infer_as_varchar': 'false'
    }
}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=nb_data_request)

data_df_1 = itcfs.read_pandas_and_concat(readClient, flightInfo, timeout=240)
data_df_1.head(10)

POSITION_CODE DEPARTMENT_CODE DAYS_WITH_COMPANY COMMUTE_TIME  \
0          1200             200              1825           29   
1          1200             200              2615            0   
2          1300             320              1609           30   
3          1300             320              2035           13   
4          1400             380              1885           31   
5          1400             380              3070           35   
6          1400             380              1825            9   
7          1500             300              2035           11   
8          1500             300              2250            0   
9          1600             330              1520           31   

  AGE_BEGIN_PERIOD GENDER_CODE ATTRITION PERIOD_TOTAL_DAYS STARTING_SALARY  \
0               55           0         0               330       159230.77   
1               49           0         0               180       181153.85   
2               44           1         0               330       129692.31   
3               45           0         0               330       146769.23   
4               44           0         0               330       146769.23   
5               40           0         0               330       157846.15   
6               40           1         0               330       146769.23   
7               46           0         0               330       132923.08   
8               45           1         0               180       150461.54   
9               46           0         0               330       143076.92   

  ENDING_SALARY  ... VACATION_DAYS_TAKEN SICK_DAYS_TAKEN PROMOTIONS  \
0     161538.46  ...                  28            10.5          0   
1     183846.15  ...                  15               4          0   
2     132923.08  ...                  20              15          0   
3     150461.54  ...                  28               8          0   
4     150461.54  ...                  26            11.5          0   
5     161538.46  ...                  19            12.5          0   
6     150461.54  ...                  20               9          0   
7     136153.85  ...                  10               9          0   
8     154153.85  ...                  10               4          0   
9     146769.23  ...                  26             9.5          0   

  NB_MANAGERS DAYS_IN_POSITION DAYS_SINCE_LAST_RAISE RANKING_CODE OVERTIME  \
0           1             1825                     0            3        0   
1           1             2615                    60            3        0   
2           1             1609                   150            3        0   
3           1             2035                   210            3        0   
4           1             1885                    60            3        0   
5           1             3070                   150            3        0   
6           1             1825                     0            3        0   
7           1             2035                   210            3        0   
8           1             2250                    60            3        0   
9           1             1520                    60            3        0   

  DBLOVERTIME TRAVEL  
0           0      0  
1           0      0  
2           0      0  
3           0      0  
4           0      0  
5           0      0  
6           0      0  
7           0      0  
8           0      0  
9           0      0  

[10 rows x 23 columns]

In [8]:
import pandas as pd

In [9]:
scoring_df = data_df_1.sample(n=ROWS_TO_SCORE).drop(['ATTRITION'], axis=1)
scoring_df = scoring_df.apply(pd.to_numeric, errors='coerce').fillna(scoring_df)

scoring_df.head()

POSITION_CODE  DEPARTMENT_CODE  DAYS_WITH_COMPANY  COMMUTE_TIME  \
218           5700              390               1700             0   
113           4600              360               1155            28   
101           4400              340               1000            18   
75            4000              300               2063            22   
210           5700              390               1245            21   

     AGE_BEGIN_PERIOD  GENDER_CODE  PERIOD_TOTAL_DAYS  STARTING_SALARY  \
218                35            0                180         60423.08   
113                56            0                330         55730.77   
101                25            1                330         29615.38   
75                 30            1                330         28153.85   
210                37            0                330         49846.15   

     ENDING_SALARY  NB_INCREASES  ...  VACATION_DAYS_TAKEN  SICK_DAYS_TAKEN  \
218       60423.08             0  ...                    4              0.0   
113       58846.15             1  ...                   15              7.5   
101       31730.77             1  ...                   24              7.0   
75        29692.31             1  ...                   14              6.0   
210       52269.23             1  ...                   25              9.0   

     PROMOTIONS  NB_MANAGERS  DAYS_IN_POSITION  DAYS_SINCE_LAST_RAISE  \
218           0            1              1700                    240   
113           0            1              1155                     60   
101           0            1              1000                    270   
75            0            1              2063                    240   
210           0            1              1245                    150   

     RANKING_CODE  OVERTIME  DBLOVERTIME  TRAVEL  
218             3     0.000        0.000       0  
113             3     0.000        0.000       0  
101             2    25.875        2.475       0  
75              3    21.250        0.000       0  
210             3     0.000        0.000       0  

[5 rows x 22 columns]

In [10]:
scoring_df.dtypes

POSITION_CODE              int64
DEPARTMENT_CODE            int64
DAYS_WITH_COMPANY          int64
COMMUTE_TIME               int64
AGE_BEGIN_PERIOD           int64
GENDER_CODE                int64
PERIOD_TOTAL_DAYS          int64
STARTING_SALARY          float64
ENDING_SALARY            float64
NB_INCREASES               int64
BONUS                    float64
NB_BONUS                   int64
VACATION_DAYS_TAKEN        int64
SICK_DAYS_TAKEN          float64
PROMOTIONS                 int64
NB_MANAGERS                int64
DAYS_IN_POSITION           int64
DAYS_SINCE_LAST_RAISE      int64
RANKING_CODE               int64
OVERTIME                 float64
DBLOVERTIME              float64
TRAVEL                     int64
dtype: object

In [11]:
fields = scoring_df.columns.tolist()
values = scoring_df[fields].values.tolist()

In [12]:
payload = {"input_data": [{"fields": fields, "values": []}]}

In [13]:
import math

for row in values:
    new_row = []
    for value in row:
        if value == math.floor(value):
            new_row.append(math.floor(value))
        else:
            new_row.append(value)
    payload["input_data"][0]["values"].append(new_row)

In [14]:
response = wml_client.deployments.score(MODEL_UID, payload)

In [15]:
response

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [[False, [0.89, 0.11]],
    [False, [0.95, 0.05]],
    [False, [0.92, 0.08]],
    [False, [0.91, 0.09]],
    [False, [0.9, 0.1]],
    [True, [0.27, 0.73]],
    [False, [0.94, 0.06]],
    [True, [0.06, 0.94]],
    [False, [0.87, 0.13]],
    [False, [0.95, 0.05]]]}]}